In [2]:
!pip install kagglehub

  Using cached kagglehub-0.3.12-py3-none-any.whl.metadata (38 kB)
Using cached kagglehub-0.3.12-py3-none-any.whl (67 kB)


In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D,TimeDistributed, Activation, Dropout, Flatten, Dense, Input, Layer
from tensorflow.keras.layers import Embedding, LSTM, add, Concatenate, Reshape, concatenate, Bidirectional
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet201, EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input as ef_preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import kagglehub
from glob import glob
from PIL import Image
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mo_preprocess_input
import random

c:\Users\user2\.conda\envs\tf-gpu\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [ ]:
# # Download latest version
# path = kagglehub.dataset_download("adityajn105/flickr8k")

# print("Path to dataset files:", path)

100%|██████████| 1.04G/1.04G [01:01<00:00, 18.2MB/s]

Extracting files...


Path to dataset files: C:\Users\user2\.cache\kagglehub\datasets\adityajn105\flickr8k\versions\1


In [ ]:
# !mkdir dataset
# !copy "C:\Users\user2\.cache\kagglehub\datasets\adityajn105\flickr8k\versions\1\*" flickr\

C:\Users\user2\.cache\kagglehub\datasets\adityajn105\flickr8k\versions\1\captions.txt
        1�� ������ ����Ǿ����ϴ�.


In [ ]:
# image 데이터 추출
for file_name in glob('C:/Users/user2/Documents/folder/github/flickr/image/*.jpg')[:5]:
    print(file_name.split('\\')[1])
    # print(np.array(Image.open(file_name)))

1000268201_693b08cb0e.jpg
1001773457_577c3a7d70.jpg
1002674143_1b742ab4b8.jpg
1003163366_44323f5815.jpg
1007129816_e794419615.jpg


In [ ]:
# target 추출

with open('flickr/captions.txt', 'r') as f:
    for line in f.readlines()[:5]:
        if 'jpg,' not in line:
            continue
        print(line.strip())

1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg,A girl going into a wooden building .
1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playhouse .


In [ ]:
# target 추출

with open('flickr/captions.txt', 'r') as f:
    for line in f.readlines()[:5]:
        if 'jpg,' not in line:
            continue
        print(line.split('jpg,')[1].strip())

A child in a pink dress is climbing up a set of stairs in an entry way .
A girl going into a wooden building .
A little girl climbing into a wooden playhouse .
A little girl climbing the stairs to her playhouse .


In [ ]:
# 파일 경로
caption_path = 'flickr/captions.txt'

# (image_filename, caption) 쌍 리스트 만들기
data_pairs = []

with open(caption_path, 'r') as f:
    for line in f.readlines():
        if 'jpg,' not in line:
            continue
        parts = line.strip().split(',', maxsplit=1)
        image_file = parts[0]
        caption = parts[1]
        data_pairs.append((image_file, caption))

In [11]:
data_pairs[:5]

[('1000268201_693b08cb0e.jpg',
  'A child in a pink dress is climbing up a set of stairs in an entry way .'),
 ('1000268201_693b08cb0e.jpg', 'A girl going into a wooden building .'),
 ('1000268201_693b08cb0e.jpg',
  'A little girl climbing into a wooden playhouse .'),
 ('1000268201_693b08cb0e.jpg',
  'A little girl climbing the stairs to her playhouse .'),
 ('1000268201_693b08cb0e.jpg',
  'A little girl in a pink dress going into a wooden cabin .')]

In [102]:
seq_len = len(y_train[0])

In [12]:
captions = [c for _, c in data_pairs]

tokenizer = Tokenizer(filters='.', lower=True, oov_token='<unk>')
tokenizer.fit_on_texts(["<start> " + c + " <end>" for c in captions])

vocab_size = len(tokenizer.word_index) + 1
max_len = max(len(c.split()) for c in captions) + 2  # <start>, <end> 포함

In [14]:
from tensorflow.keras.utils import Sequence

class ImageCaptionGenerator(Sequence):
    def __init__(self, data_pairs, tokenizer, image_dir,
                 batch_size=32, input_size=(224, 224), max_len=30, **kwargs):
        super().__init__(**kwargs)
        self.data_pairs = data_pairs
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.input_size = input_size
        self.max_len = max_len
        self.image_dir = image_dir

    def __len__(self):
        return len(self.data_pairs) // self.batch_size

    def __getitem__(self, idx):
        batch_pairs = random.sample(self.data_pairs, self.batch_size)

        image_batch = []
        decoder_input_batch = []
        decoder_target_batch = []

        for img_file, caption in batch_pairs:
            # 이미지 처리
            img_path = os.path.join(self.image_dir, img_file)
            img = load_img(img_path, target_size=self.input_size)
            img_array = img_to_array(img)
            img_array = mo_preprocess_input(img_array)
            image_batch.append(img_array)

            # 캡션 처리
            seq = self.tokenizer.texts_to_sequences(["<start> " + caption + " <end>"])[0]
            dec_input = seq[:-1]
            dec_target = seq[1:]

            # 패딩
            dec_input = pad_sequences([dec_input], maxlen=self.max_len, padding='post')[0]
            dec_target = pad_sequences([dec_target], maxlen=self.max_len, padding='post')[0]

            decoder_input_batch.append(dec_input)
            decoder_target_batch.append(dec_target)

        return (np.array(image_batch), np.array(decoder_input_batch)), np.array(decoder_target_batch)

In [ ]:
image_dir = 'flickr/image/'

asdf = ImageCaptionGenerator(data_pairs,
                             tokenizer,
                             image_dir,
                             batch_size=32,
                             input_size=(224, 224),
                             max_len=max_len)

In [33]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_pairs, test_size=0.1)

In [34]:
len(train_data), len(test_data)

(36409, 4046)

In [35]:
train_data, val_data = train_test_split(train_data, test_size=0.1)

In [36]:
len(train_data), len(val_data)

(32768, 3641)

In [37]:
import tensorflow as tf

mobile_net = tf.keras.applications.MobileNetV2(weights='imagenet')
mobile_net.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [38]:
mobile_input = mobile_net.layers[0].output
mobile_output = mobile_net.layers[154].output

mobile = Model(inputs=mobile_input, outputs=mobile_output)

mobile.trainable = False

en_train_dense_1 = Dense(256, activation='relu')
en_train_dense_1_out = en_train_dense_1(mobile_output)

h_dense = Dense(128, activation='tanh')(en_train_dense_1_out)
c_dense = Dense(128, activation='tanh')(en_train_dense_1_out)

train_lstm = [h_dense, c_dense]

en_model = Model(inputs=mobile_input, outputs=train_lstm)

en_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [39]:
de_input = Input(shape=(max_len,))

de_embed = Embedding(input_dim=vocab_size, output_dim=64, mask_zero=True)
de_embed_out = de_embed(de_input)

de_lstm = LSTM(units=128, return_sequences=True, return_state=True)
de_lstm_out, de_lstm_h, de_lstm_c = de_lstm(de_embed_out, initial_state=train_lstm)

de_dense = Dense(vocab_size, activation='softmax')
de_outputs = de_dense(de_lstm_out)

decoder = Model(inputs=[train_lstm, de_input], outputs=de_outputs)
decoder.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 40)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 40, 64)       587648      ['input_6[0][0]']                
                                                                                                  
 input_7 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 128)]        0           []                               
                                                                                            

In [40]:
img_test_model = Model(inputs=[en_model.input,de_input], outputs=de_outputs)
img_test_model.output

<KerasTensor: shape=(None, 40, 9182) dtype=float32 (created by layer 'dense_7')>

In [41]:
img_test_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [42]:
img_test_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [43]:
train_data_gen = ImageCaptionGenerator(data_pairs=train_data,
                                       tokenizer=tokenizer,
                                       image_dir=image_dir,
                                       batch_size=32,
                                       max_len=max_len)
val_data_gen = ImageCaptionGenerator(data_pairs=val_data,
                                       tokenizer=tokenizer,
                                       image_dir=image_dir,
                                       batch_size=32,
                                       max_len=max_len)
test_data_gen = ImageCaptionGenerator(data_pairs=test_data,
                                       tokenizer=tokenizer,
                                       image_dir=image_dir,
                                       batch_size=32,
                                       max_len=max_len)

In [45]:
hist = img_test_model.fit(train_data_gen,
                          callbacks=[EarlyStopping(monitor='val_loss',patience=5),
                                     ModelCheckpoint(filepath='lfw_model_1.keras', monitor='val_loss', save_best_only=True, verbose=1)],
                          epochs=100,
                          validation_data=val_data_gen)

Epoch 1/100
1024/1024 [==============================] - ETA: 0s - loss: 0.8288 - accuracy: 0.4030
Epoch 1: val_loss improved from inf to 1.01599, saving model to lfw_model_1.keras
1024/1024 [==============================] - 138s 135ms/step - loss: 0.8288 - accuracy: 0.4030 - val_loss: 1.0160 - val_accuracy: 0.3731
Epoch 2/100
1024/1024 [==============================] - ETA: 0s - loss: 0.8176 - accuracy: 0.4068
Epoch 2: val_loss improved from 1.01599 to 1.00106, saving model to lfw_model_1.keras
1024/1024 [==============================] - 143s 139ms/step - loss: 0.8176 - accuracy: 0.4068 - val_loss: 1.0011 - val_accuracy: 0.3709
Epoch 3/100
1024/1024 [==============================] - ETA: 0s - loss: 0.8047 - accuracy: 0.4121
Epoch 3: val_loss did not improve from 1.00106
1024/1024 [==============================] - 139s 136ms/step - loss: 0.8047 - accuracy: 0.4121 - val_loss: 1.0025 - val_accuracy: 0.3725
Epoch 4/100
1024/1024 [==============================] - ETA: 0s - loss: 0.79